<a href="https://colab.research.google.com/github/plakhtiizlata-dotcom/robotics_algorithms/blob/master/Traffic_and_Road_Signsipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
from google.colab import drive

drive.mount('/content/drive')

!pip install ultralytics roboflow -q

import ultralytics
from ultralytics import YOLO
from roboflow import Roboflow

ultralytics.checks()

Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.6/112.6 GB disk)


In [ ]:

rf = Roboflow(api_key="wNVNT00XBBZbek6CJIeW")

project = rf.workspace("usmanchaudhry622-gmail-com").project("traffic-and-road-signs")
dataset = project.version(1).download("yolov8")

print(f"Dataset downloaded to: {dataset.location}")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Traffic-and-Road-Signs-1 in yolov8:: 100%|██████████| 20012/20012 [00:07<00:00, 2841.20it/s]


Dataset downloaded to: /content/Traffic-and-Road-Signs-1


In [ ]:
# Ініціалізація та запуск навчання
model = YOLO('yolov8n.pt')

# Запуск тренування
model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=25,
    imgsz=640,
    batch=16,
    name='yolo_traffic_lab',
    plots=True
)

Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Traffic-and-Road-Signs-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_traffic_lab, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspecti

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d1edd558620>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,   

In [ ]:
# Валідація найкращої моделі
best_model = YOLO('/content/runs/detect/yolo_traffic_lab/weights/best.pt')
metrics = best_model.val()

print("\n--- Metrics ---")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")

Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,011,303 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 199.7±80.8 MB/s, size: 8.8 KB)
val: Scanning /content/Traffic-and-Road-Signs-1/valid/labels.cache... 1884 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1884/1884 2.2Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 118/118 6.5it/s 18.1s
                   all       1884       1886      0.939      0.937      0.943      0.803
-Road narrows on right         54         54      0.998          1      0.995      0.894
    50 mph speed limit         62         62      0.528      0.984      0.732      0.594
     Attention Please-        117        117          1      0.996      0.995      0.858
    Beware of children        106        106          1      0.992      0.995      0.889
CYCLE ROUTE AHEAD WARNING     

In [14]:
import shutil
import datetime

# Створення архіву з результатами
output_filename = f"yolo_lab_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}"
shutil.make_archive(output_filename, 'zip', '/content/runs/detect/yolo_traffic_lab')

# Копіювання на Google Drive
destination = f"/content/drive/MyDrive/{output_filename}.zip"
shutil.move(f"{output_filename}.zip", destination)

print(f"Results saved to Google Drive: {destination}")

Results saved to Google Drive: /content/drive/MyDrive/yolo_lab_results_20251202_1037.zip
